In [1]:
import whisper
from pyannote.audio import Pipeline
import gradio as gr
import networkx as nx
import json
import os
from dotenv import load_dotenv
import google.generativeai as genai

C:\Users\aryal\anaconda\envs\llms\Lib\site-packages\pyannote\audio\core\io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()


In [ ]:
whisper_model = whisper.load_model("medium")
diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
def transcribe_audio(audio_path):
    result = whisper_model.transcribe(audio_path)
    return result['text']

In [ ]:
def diarize_audio(audio_path):
    diarization = diarization_pipeline(audio_path)
    speakers = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        speakers.append({
            "start": turn.start,
            "end": turn.end,
            "speaker": speaker
        })
    return speakers

In [ ]:
def extract_decisions_actions(transcript):
    prompt = f"""
    Extract all decisions and action items from the following meeting transcript:
    {transcript}
    Provide a JSON with keys: "decisions" and "action_items".
    """
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response.choices[0].message.content

In [ ]:
def build_cytoscape_elements(data):
    elements = []
    # Add decision nodes
    for i, decision in enumerate(data.get("decisions", [])):
        elements.append({
            'data': {'id': f"decision_{i}", 'label': decision, 'type': 'decision'}
        })
    # Add action nodes and edges
    for j, action in enumerate(data.get("action_items", [])):
        elements.append({
            'data': {'id': f"action_{j}", 'label': action, 'type': 'action_item'}
        })
        # Link to first decision (for simplicity)
        if data.get("decisions"):
            elements.append({
                'data': {
                    'source': f"decision_0",
                    'target': f"action_{j}"
                }
            })
    return elements

In [ ]:
def process_meeting(audio_file):
    transcript = transcribe_audio(audio_file)
    speakers = diarize_audio(audio_file)
    decisions_actions_json = extract_decisions_actions(transcript)
    
    # Convert JSON string to dict
    try:
        decisions_actions = json.loads(decisions_actions_json)
    except:
        decisions_actions = {"decisions": [], "action_items": []}
    
    graph_html = build_cytoscape_html(decisions_actions)
    return transcript, speakers, decisions_actions, graph_html


In [ ]:
cyto_style = [
    {
        "selector": "node[type='decision']",
        "style": {"background-color": "skyblue", "label": "data(label)", "font-size": 12}
    },
    {
        "selector": "node[type='action_item']",
        "style": {"background-color": "lightgreen", "label": "data(label)", "font-size": 12}
    },
    {
        "selector": "edge",
        "style": {"line-color": "#ccc", "target-arrow-shape": "triangle"}
    }
]

In [ ]:
def build_cytoscape_html(data):
    elements = []
    for i, decision in enumerate(data.get("decisions", [])):
        elements.append({"data": {"id": f"decision_{i}", "label": decision}})
    for j, action in enumerate(data.get("action_items", [])):
        elements.append({"data": {"id": f"action_{j}", "label": action}})
        if data.get("decisions"):
            elements.append({"data": {"source": "decision_0", "target": f"action_{j}"}})
    
    # Convert to JSON string
    elements_json = json.dumps(elements)
    
    html_code = f"""
    <div id="cy" style="width: 100%; height: 500px;"></div>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/cytoscape/3.25.0/cytoscape.min.js"></script>
    <script>
        var cy = cytoscape({{
            container: document.getElementById('cy'),
            elements: {elements_json},
            style: [
                {{ selector: 'node', style: {{ 'label': 'data(label)', 'background-color': '#60A5FA' }} }},
                {{ selector: 'edge', style: {{ 'line-color': '#ccc', 'target-arrow-shape': 'triangle', 'width': 2 }} }}
            ],
            layout: {{ name: 'cose' }}
        }});
    </script>
    """
    return html_code


In [ ]:
import gradio as gr

# Custom CSS for styling
custom_css = """
body {background-color: #f0f4f8; font-family: 'Arial', sans-serif;}
.gradio-container {border-radius: 15px; padding: 20px; box-shadow: 0 10px 25px rgba(0,0,0,0.1);}
h1 {color: #ff4b2b; text-align: center;}
.gr-button {background-color: #ff4b2b; color: white; border-radius: 8px; padding: 10px 20px;}
.gr-textbox, .gr-json, .gr-html {border-radius: 10px; padding: 10px; background-color: #ffffff;}
.gr-textbox textarea {font-family: 'Courier New', monospace; font-size: 14px;}
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Base()) as iface:
    gr.Markdown("<h1>🚀 Smart Meeting Synthesizer</h1>")
    gr.Markdown("<p style='text-align:center;'>Upload your meeting audio to get transcripts, speaker info, decisions, and an interactive knowledge graph!</p>")
    
    with gr.Column(scale=1):
        audio_input = gr.Audio(type="filepath", label="Upload Meeting Audio")
    with gr.Column(scale=2):
        tabs = gr.Tabs()
        with tabs:
            with gr.TabItem("Transcript 📜"):
                transcript_output = gr.Textbox(label="Transcript", lines=10)
            with gr.TabItem("Speakers 🗣️"):
                speakers_output = gr.JSON(label="Speakers")
            with gr.TabItem("Decisions & Actions ✅"):
                actions_output = gr.JSON(label="Decisions & Action Items")
            with gr.TabItem("Knowledge Graph 🌐"):
                graph_output = gr.HTML(label="Knowledge Graph")
    
    submit_btn = gr.Button("Process Meeting", variant="primary")
    
    submit_btn.click(
        fn=process_meeting,
        inputs=audio_input,
        outputs=[transcript_output, speakers_output, actions_output, graph_output]
    )

iface.launch()
